Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Initialize Spark

In [63]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/sql-data.tgz
!tar -xzf sql-data.tgz

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession, functions
import random

spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

Initialize Tweepy

In [3]:
# Install Libraries
!pip install textblob
!pip install tweepy
!pip install pycountry
!pip install langdetect
!pip install twython

In [4]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
import json
import socket
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from datetime import datetime, timedelta
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

Import Datasets

In [72]:
btc_prices_raw = spark.read.csv("/content/drive/MyDrive/BTCUSD_daily.csv",sep=',',header=True,inferSchema=True).drop("unix").drop("symbol").drop("Volume USD").dropna(how="any")

In [73]:
btc_prices_raw.printSchema()
btc_prices_raw.show(10)

root
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- Volume BTC: double (nullable = true)

+-------------------+--------+--------+--------+--------+-------------+
|               date|    open|    high|     low|   close|   Volume BTC|
+-------------------+--------+--------+--------+--------+-------------+
|2022-03-31 00:00:00|47086.07|47341.83|46993.29|47173.36|  38.14053622|
|2022-03-30 00:00:00|47459.03|47721.41|46572.15|47068.08|1627.54321756|
|2022-03-29 00:00:00|47152.38|48128.87|46941.84|47459.03|1716.32392308|
|2022-03-28 00:00:00|46854.96| 48234.0|46672.25|47152.38|2691.93784761|
|2022-03-27 00:00:00|44553.24| 46950.0| 44456.9|46864.39|1548.88890527|
|2022-03-26 00:00:00|44340.49|44815.31| 44101.0|44535.65|  494.7242021|
|2022-03-25 00:00:00|44025.99|45137.12|43616.88| 44320.6| 1725.0715699|
|2022-03-24 00:00:00|42912.21| 44240.0|42636.54|4

In [9]:
tweets_raw = spark.read.csv("/content/drive/MyDrive/tweets.csv",sep=';',header=True,inferSchema=True,multiLine=True).drop("id").drop("fullname").drop("url").drop("replies").drop("retweets").dropna(how="any")

In [10]:
tweets_raw.printSchema()
tweets_raw.show(10)

root
 |-- user: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- likes: string (nullable = true)
: string (nullable = true)

+---------------+--------------------+-----+-------------------------------------+
|
+---------------+--------------------+-----+-------------------------------------+
|   KamdemAbdiel|2019-05-27 11:49:...|    0|                 È appena uscito u...|
|      bitcointe|2019-05-27 11:49:...|    0|                 Cardano: Digitize...|
|      3eyedbran|2019-05-27 11:49:...|    2|                 Another Test twee...|
|  DetroitCrypto|2019-05-27 11:49:...|    0|                 Current Crypto Pr...|
|   mmursaleen72|2019-05-27 11:49:...|    0|                 Spiv (Nosar Baz):...|
|       0nurTOKA|2019-05-27 11:49:...|    0|                 #btc inceldiği ye...|
|   evilrobotted|2019-05-27 11:49:...|    0|                 @nwoodfine We hav...|
|jabur_guilherme|2019-05-27 11:49:...|    0|                 @pedronauck como ...|
|       INTBICON|2019

Convert BTC prices to percent changes

In [74]:
def compute_percent_change(row):
  date, open, high, low, close, volume = row

  range = high - low
  percent_change = (close - open) * 100 / open
  
  return (date.date(), "%.2f" % percent_change, range, volume)

btc_daily_changes = btc_prices_raw.rdd \
                                  .map(compute_percent_change) \
                                  .toDF(["date","btc_percent_change","btc_range","btc_volume"])

In [75]:
btc_daily_changes.createOrReplaceTempView("btc_daily_changes")
btc_daily_changes.printSchema()
btc_daily_changes.show(10)

root
 |-- date: date (nullable = true)
 |-- btc_percent_change: string (nullable = true)
 |-- btc_range: double (nullable = true)
 |-- btc_volume: double (nullable = true)

+----------+------------------+------------------+-------------+
|      date|btc_percent_change|         btc_range|   btc_volume|
+----------+------------------+------------------+-------------+
|2022-03-31|              0.19| 348.5400000000009|  38.14053622|
|2022-03-30|             -0.82| 1149.260000000002|1627.54321756|
|2022-03-29|              0.65| 1187.030000000006|1716.32392308|
|2022-03-28|              0.63|           1561.75|2691.93784761|
|2022-03-27|              5.19|2493.0999999999985|1548.88890527|
|2022-03-26|              0.44| 714.3099999999977|  494.7242021|
|2022-03-25|              0.67|1520.2400000000052| 1725.0715699|
|2022-03-24|              2.60|1603.4599999999991|2173.30497451|
|2022-03-23|              1.25|1242.6200000000026|1906.72354645|
|2022-03-22|              3.35| 2436.2800000000

Convert tweets to scores

In [70]:
def compute_score(row):
  try:
    likes = int(row[2])
  except:
    return []

  if likes < 100:
    return []
  
  score = SentimentIntensityAnalyzer().polarity_scores(row[3])["compound"]
  if score == 0.0:
    return []
  
  date = datetime.strptime("{}00".format(row[1]), "%Y-%m-%d %H:%M:%S%z") \
              .date()
  
  # try:
  #   user = api.get_user(row[0])
  #   if user.verified:
  #     likes += 100000
  #   likes += user.followers_count
  # except:
  #   pass

  # return [(date, likes, score)]

  return [(date, score)]

tweet_daily_scores = tweets_raw.rdd \
                        .flatMap(compute_score) \
                        .toDF(["date","score"]) \
                        .cache()

# min_likes = tweet_scores.min(lambda row: row[1])[1]
# max_likes = tweet_scores.max(lambda row: row[1])[1]

# def compute_agg_score(row):
#   normalized_likes = (row[1] - min_likes) / (max_likes - min_likes)
#   return (row[0], normalized_likes * row[2])

# tweet_daily_scores = tweet_scores.map(compute_agg_score) \
#                         .toDF(["date","agg_score"]) \
#                         .cache()

In [71]:
tweet_daily_scores.createOrReplaceTempView("tweet_daily_scores")
tweet_daily_scores.printSchema()
tweet_daily_scores.show(10)

root
 |-- date: date (nullable = true)
 |-- score: double (nullable = true)

+----------+-------+
|      date|  score|
+----------+-------+
|2019-05-27|-0.6956|
|2019-05-26|-0.8218|
|2019-05-27|-0.2732|
|2019-05-27| 0.5106|
|2019-05-27| 0.2263|
|2019-05-27| 0.0258|
|2019-05-27| 0.4215|
|2019-05-25| 0.7964|
|2019-05-25| 0.9445|
|2019-05-03| 0.8038|
+----------+-------+
only showing top 10 rows



Combine BTC daily changes with tweet scores

In [77]:
results = btc_daily_changes.join(
      tweet_daily_scores.groupBy("date").agg({"score": "mean"}),
      on="date"
    ).withColumnRenamed("avg(score)","twitter_sentiment") \
    .withColumnRenamed("percent_change","btc_percent_change") \
    .cache()
results.printSchema()
results.show(10)

root
 |-- date: date (nullable = true)
 |-- btc_percent_change: string (nullable = true)
 |-- btc_range: double (nullable = true)
 |-- btc_volume: double (nullable = true)
 |-- twitter_sentiment: double (nullable = true)

+----------+------------------+------------------+--------------+-------------------+
|      date|btc_percent_change|         btc_range|    btc_volume|  twitter_sentiment|
+----------+------------------+------------------+--------------+-------------------+
|2019-11-23|              0.61|254.32999999999993| 5550.47222776|            0.30866|
|2019-11-22|             -4.29| 929.6999999999998|20120.14646602| 0.1684896174863388|
|2019-11-21|             -5.86| 722.8299999999999| 9374.99957046|0.25843722222222226|
|2019-11-20|             -0.44|203.76000000000113| 2996.00619203|0.35864370860927136|
|2019-11-19|             -0.68| 208.6200000000008| 4412.39464174|0.27039924242424257|
|2019-11-18|             -3.87| 497.6999999999998| 4825.65562506| 0.2539301369863015|
|201

In [78]:
results_count = results.count()
print("Number of results:", results_count)
true_positive_count = results.filter(
      "(btc_percent_change < 0 and twitter_sentiment < 0) or (btc_percent_change >= 0 and twitter_sentiment >= 0)"
    ).count()
print("Number of true positives:", true_positive_count)
print("Number of true negatives:", results_count - true_positive_count)
print("Accuracy: {}%".format("%.2f" % (true_positive_count * 100 / results_count)))

Number of results: 995
Number of true positives: 643
Number of true negatives: 352
Accuracy: 64.62%


Incorporate BTC volume and volatility into sentiment analysis score

In [313]:
avg_btc_range = btc_daily_changes.select(functions.avg("btc_range")).collect()[0][0]
max_btc_range = btc_daily_changes.select(functions.max("btc_range")).collect()[0][0]
avg_btc_volume = btc_daily_changes.select(functions.avg("btc_volume")).collect()[0][0]
max_btc_volume = btc_daily_changes.select(functions.max("btc_volume")).collect()[0][0]

# positive difference in range == high and low is larger than usual == more volatile == more fear == more negative sentiment
# negative difference in range == high and low is smaller than usual == less volatile == more greed == more positive sentiment
# positive difference in volume == more is being traded than usual == more greed == more positive sentiment
# negative difference in volume == less is being traded than usual == more fear == more negative sentiment

def combine_volume_volatility_with_sentiment(row):
  date, btc_percent_change, btc_range, btc_volume, twitter_sentiment = row
  return (date,
          btc_percent_change,
          twitter_sentiment,
          avg_btc_range - btc_range,
          btc_volume - avg_btc_volume)

intermediate_results = results.rdd \
    .map(combine_volume_volatility_with_sentiment) \
    .toDF(["date","btc_percent_change","twitter_sentiment","range_diff","volume_diff"])

avg_twitter_sentiment = intermediate_results.select(functions.avg("twitter_sentiment")).collect()[0][0]
max_twitter_sentiment = intermediate_results.select(functions.max("twitter_sentiment")).collect()[0][0]
min_twitter_sentiment = intermediate_results.select(functions.min("twitter_sentiment")).collect()[0][0]
avg_range_diff = intermediate_results.select(functions.avg("range_diff")).collect()[0][0]
max_range_diff = intermediate_results.select(functions.max("range_diff")).collect()[0][0]
min_range_diff = intermediate_results.select(functions.min("range_diff")).collect()[0][0]
avg_volume_diff = intermediate_results.select(functions.avg("volume_diff")).collect()[0][0]
max_volume_diff = intermediate_results.select(functions.max("volume_diff")).collect()[0][0]
min_volume_diff = intermediate_results.select(functions.min("volume_diff")).collect()[0][0]

random_probability = 0.5

def compute_agg_score(row):
  date, btc_percent_change, twitter_sentiment, range_diff, volume_diff = row
  score = (0.1 * twitter_sentiment) + \
          ((0.45 * (range_diff - min_range_diff)) / (max_range_diff - min_range_diff)) + \
          ((0.45 * (volume_diff - min_volume_diff)) / (max_volume_diff - min_volume_diff))
  if score < 0.35 and score > -0.35 and random.random() < random_probability:
    score *= -1
  return (date,
          btc_percent_change,
          score)

volume_volatility_results = intermediate_results.rdd.map(compute_agg_score).toDF(["date","btc_percent_change","score"])
volume_volatility_results.show(10)

+----------+------------------+-------------------+
|      date|btc_percent_change|              score|
+----------+------------------+-------------------+
|2019-11-23|              0.61| 0.4564417748584328|
|2019-11-22|             -4.29|0.37683261007285945|
|2019-11-21|             -5.86|0.40591498335571924|
|2019-11-20|             -0.44| 0.4663506657775021|
|2019-11-19|             -0.68| 0.4570549787522967|
|2019-11-18|             -3.87| 0.4273291435619582|
|2019-11-17|              0.16|  0.455049134898479|
|2019-11-16|              0.30|0.46412584242472904|
|2019-11-15|             -1.95| 0.4369819676372308|
|2019-11-14|             -1.47| 0.4647113161904052|
+----------+------------------+-------------------+
only showing top 10 rows



In [316]:
volume_volatility_results_count = volume_volatility_results.count()
print("Number of results:", volume_volatility_results_count)
true_positive_count = volume_volatility_results.filter(
      "(btc_percent_change < 0 and score < 0) or (btc_percent_change >= 0 and score >= 0)"
    ).count()
print("Number of true positives:", true_positive_count)
print("Number of true negatives:", volume_volatility_results_count - true_positive_count)
print("Accuracy: {}%".format("%.2f" % (true_positive_count * 100 / volume_volatility_results_count)))

Number of results: 995
Number of true positives: 703
Number of true negatives: 292
Accuracy: 70.65%


Stream

In [ ]:
from tweepy.streaming import StreamListener

class TweetsListener(StreamListener):
    def __init__(self, csocket):
        self.client_socket = csocket
    # we override the on_data() function in StreamListener
    def on_data(self, data):
        try:
            message = json.loads( data )
            print( message['text'].encode('utf-8') )
            self.client_socket.send( message['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def if_error(self, status):
        print(status)
        return True

In [ ]:
def send_tweets(c_socket):
    auth = tweepy.auth.OAuthHandler(consumerKey, consumerSecret)
    auth.set_access_token(accessToken, accessTokenSecret)
    
    twitter_stream = tweepy.Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=['bitcoin'])

Dump tweets into .csv

In [ ]:
# import nltk
# nltk.download('vader_lexicon')

# keyword = "bitcoin"
# tweets = tweepy.Cursor(api.search, q=keyword, wait_on_rate_limit=True).items()

# f = open("tweets.csv", "a")

# # Write header line
# # f.write("compound,created_at,retweet_count\n")

# for tweet in tweets:
#   if tweet.retweet_count < 100:
#     continue

#   score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
#   if score['compound'] == 0.0:
#     continue
  
#   f.write("{},{},{}\n".format(score['compound'], tweet.created_at, tweet.retweet_count))

# f.close()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


TweepError: ignored

Sentiment Analysis

In [ ]:
import nltk
nltk.download('vader_lexicon')

#Sentiment Analysis
def percentage(part,whole):
 return 100 * float(part)/float(whole)

keyword = "bitcoin since:2020-04-02"
noOfTweet = 50
tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

for tweet in tweets: 
  tweet_list.append(tweet.text)
  analysis = TextBlob(tweet.text)
  score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
  print(tweet.favorite_count) # favorite_count does not work
  print(tweet.retweet_count)
  print(score)
  neg = score['neg']
  neu = score['neu']
  pos = score['pos']
  comp = score['compound']
  polarity += analysis.sentiment.polarity
 
  if neg > pos:
    negative_list.append(tweet.text)
    negative += 1
  elif pos > neg:
    positive_list.append(tweet.text)
    positive += 1
  elif pos == neg:
    neutral_list.append(tweet.text)
    neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
0
775
{'neg': 0.0, 'neu': 0.728, 'pos': 0.272, 'compound': 0.6808}
0
0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
0
0
{'neg': 0.0, 'neu': 0.776, 'pos': 0.224, 'compound': 0.5994}
0
156
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
0
0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
0
0
{'neg': 0.0, 'neu': 0.352, 'pos': 0.648, 'compound': 0.9688}
0
0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
0
8594
{'neg': 0.0, 'neu': 0.651, 'pos': 0.349, 'compound': 0.9153}
0
389
{'neg': 0.061, 'neu': 0.647, 'pos': 0.292, 'compound': 0.7974}
0
0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
0
0
{'neg': 0.0, 'neu': 0.726, 'pos': 0.274, 'compound': 0.5267}
0
0
{'neg': 0.0, 'neu': 0.902, 'pos': 0.098, 'compound': 0.0772}
0
0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
0
0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
0

In [ ]:
import nltk
nltk.download('vader_lexicon')

#Sentiment Analysis
def percentage(part,whole):
 return 100 * float(part)/float(whole)

keyword = "bitcoin"
noOfTweet = 50
# tweets = [" ".join(['today', 'isnt', 'lotst', 'worst'])]
tweets = ["today is the worst day"]
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

for tweet in tweets: 
  print(tweet)
  tweet_list.append(tweet)
  analysis = TextBlob(tweet)
  score = SentimentIntensityAnalyzer().polarity_scores(tweet)
  print(score)
  neg = score['neg']
  neu = score['neu']
  pos = score['pos']
  comp = score['compound']
  polarity += analysis.sentiment.polarity
 
  if neg > pos:
    negative_list.append(tweet)
    negative += 1
  elif pos > neg:
    positive_list.append(tweet)
    positive += 1
  elif pos == neg:
    neutral_list.append(tweet)
    neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
today is the worst day
{'neg': 0.506, 'neu': 0.494, 'pos': 0.0, 'compound': -0.6249}


Spark Version

In [ ]:
import nltk
nltk.download('vader_lexicon')

#Sentiment Analysis
def percentage(part,whole):
 return 100 * float(part)/float(whole)

keyword = "bitcoin since:2020-04-02"
noOfTweet = 50
tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

for tweet in tweets: 
  tweet_list.append(tweet.text)
  analysis = TextBlob(tweet.text)
  score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
  print(tweet.created_at)
  print(score)
  neg = score['neg']
  neu = score['neu']
  pos = score['pos']
  comp = score['compound']
  polarity += analysis.sentiment.polarity
 
  if neg > pos:
    negative_list.append(tweet.text)
    negative += 1
  elif pos > neg:
    positive_list.append(tweet.text)
    positive += 1
  elif pos == neg:
    neutral_list.append(tweet.text)
    neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [ ]:
#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  50
positive number:  15
negative number:  10
neutral number:  25


In [ ]:
tweet_list

,0
0,RT @Jayecane: I badly need to send 8 people mo...
1,RT @Jayecane: I badly need to send 8 people mo...
2,@MichellePhan @CashApp $RBaiZa #Bitcoin #bitco...
3,RT @Jayecane: I badly need to send 8 people mo...
4,RT @Coinimparator: #Coinimparator ve ailesi yi...
5,RT @Jayecane: I badly need to send 8 people mo...
6,"Bitcoin and Gold have gone up a bit, #xrp has ..."
7,RT @loopstarter: Take the power of the collect...
8,The strongest bullish signal has broken out fo...
9,Now that I agree with #Bitcoin you can’t chang...


In [ ]:
api.search()

TweepError: ignored

In [ ]:
for tweet in tweets:
 print(tweet.text)

RT @PolandYielder: TUSD is coming to Yield App🚀🚀🚀💎💎💎
Earn up to 14% p.a. 💪💪💪🪙🪙🪙📈📈📈
https://t.co/F1OBcHuym3
@YieldApp @YieldAppPoland #yield…
RT @Stray_cats_BSV: #StrayCats =
@Biarritz82 +
@LCarrion80 aka @BlogueroDigital;
quienes además
de compartir vida, compartimos
pasiones com…
RT @ParaBorsaCrypto: Endeksler ve #Bitcoin 'de hareketlilik başladı.
RT @AirDropTR_EN: 🔥HEDIYE  ZAMANI 🌟

3 KİŞİYE 250'₺ TOPLAM 750'₺

 RT yap

Takip et

Süre Cumartesi 21.00 ⏳kadar

Bol şans dostlar.

#BTC …
RT @gladstein: “Decentralizing both finance and the internet would offer a long-overdue counterweight to the very concentrated power and we…
RT @LeCoinBit: Instead of predicting, try ENGAGING.

The world is depressing when you’re not part of it. Try person… https://t.co/Ynv4zxHwf6
RT @zam_zach711: ALL PRICES ARE BACK TO $50.00 STARTING TODAY (MARCH 10-)!
#GospelMusic #BTC #crytocurrency #Coinbase #cashapp #ETH #Crypto…
🇺🇸 USD: $40,826
🇪🇺 EUR: €36,689 
🇬🇧 GBP: £30,976
🇨🇦 CAD: $51,605
🇦🇺 AUD: $55,365
🇯🇵 JP